In [2]:
import pandas as pd

In [3]:
import os
%pwd

'c:\\code\\ML\\breast_cancer\\research'

In [4]:
os.chdir("../")

In [5]:
pwd

'c:\\code\\ML\\breast_cancer'

In [6]:
df = pd.read_csv("artifacts/data_ingestion/data.csv",delimiter = ",")


In [7]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [8]:
#entity

In [9]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
@dataclass
class EDAconfig:
    data_path : Path
    report_path : Path
    


In [19]:
# src / config / configuration

In [12]:
import sys
import os

sys.path.append(os.path.join(os.getcwd(), "src"))


In [13]:
from breast_cancer.constants import  *

from breast_cancer.utils.common import read_yaml , create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    # def get_data_ingestion_config(self) -> DataIngestionConfig:
    #     config = self.config.data_ingestion
    #     create_directories([config.root_dir])

    #     data_ingestion_config = DataIngestionConfig(
    #         root_dir= config.root_dir,
    #         source_url= config.source_url,
    #         local_data_file= config.local_data_file,
            
    #     )
    #     return data_ingestion_config

    

    def EDA_configuration(self) -> EDAconfig:
        config = self.config.EDA_config
        create_directories([config.report_path])

        eda_config = EDAconfig(data_path= config.data_path,report_path=config.report_path)

        return eda_config


    

In [25]:
#components

In [26]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [28]:
df.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [29]:
# we need to drop id,unnamed: 32 columns

In [ ]:
class EDA :
    def __int__(self,config:EDAconfig):
        self.config = config
    
    def load_data(self):
        return pd.read_csv(self.config.data_path)
    
    def perform_eda(self):
        df = self.load_data()

        # os.makedirs(self.config.report_path,exist_ok=True)

        #basic_info
        with open(os.path.join(self.config.report_path,"basic_info.txt"),"w") as f:
            f.write(str(df.info()))
            f.write("\n\n")
            f.write(str(df.describe()))
        
         # Null values
        df.isnull().sum().to_csv(os.path.join(self.config.report_path, "null_values.csv"))

        #
        
        
    

